In [1]:
import sys
sys.path.append("C:\\Users\\rbj428\\Documents\\research_projects\\iterative_simulation_framework\\")

### Simulation

In [13]:
import json
import sys
import re

def MakeBool(text):
    text = text.lower()
    if re.match(r'\s*true\s*', text):
        return True
    elif re.match(r'\s*false\s*', text):
        return False

def PrepareSimulation():
    from xml_extractor import xml_extractor 
    from potentials import potential_combiner

    #read in the initialization data
    with open('../potential_specs__params_state.json') as data_file:    
        potential_specs__params_state = json.load(data_file)
        potential_specs = potential_specs__params_state['specs']
        params_state = potential_specs__params_state['state']

    #create the potential
    potential = potential_combiner.Potential(potential_specs)
    potential.SetParamsState(params_state)

    #read in potential parameter values
    with open('./params_val.json') as data_file:    
        params_val = json.load(data_file)

    #load in xml settings
    xml_extractor = xml_extractor.XMLExtractor()
    xml_extractor.Parse('../settings.xml')

    ###########################################################################################

    simulation_program = xml_extractor.GetText('simulation', 'program').strip().lower()
    if simulation_program == 'gromacs':
        from gromacs_interface_tools import gromacs_potential_maker
        xmlParser = lambda x1, x2: xml_extractor.GetText('simulation', 'gromacs', x1, x2)

        #generate gromacs data
        gromacs_potential_maker = gromacs_potential_maker.SimulationPotentialConverter()
        gromacs_potential_maker.LoadPotential(potential, params_val)
        gromacs_potential_maker.TabulatePotential(r_max=float(xmlParser('table', 'r_max')), 
                                                  dr=float(xmlParser('table', 'dr'))
                                                 )
        gromacs_potential_maker.CutShiftTabulated(e_max=float(xmlParser('potential', 'e_max')), 
                                                  f_max=float(xmlParser('potential', 'f_max')), 
                                                  r_max=float(xmlParser('potential', 'r_max')), 
                                                  shift=MakeBool(xmlParser('potential', 'shift'))
                                                 )
        gromacs_potential_maker.MakeTable(filename='./table.xvg')
        gromacs_potential_maker.InsertGromppCutoff(r_buffer=float(xmlParser('grompp', 'r_buffer')), 
                                                   filename='./grompp.mdp')
    else:
        raise Exception('Invalid simulation program provided in settings file.')

In [2]:
from interface import prepare_simulation

In [5]:
prepare_simulation.PrepareSimulation()

### Update

In [7]:
import json
import sys
import re

def GenerateUpdate():
    from xml_extractor import xml_extractor 
    from potentials import potential_combiner

    #read in the initialization data
    with open('../potential_specs__params_state.json') as data_file:    
        potential_specs__params_state = json.load(data_file)
        potential_specs = potential_specs__params_state['specs']
        params_state = potential_specs__params_state['state']

    #create the potential
    potential = potential_combiner.Potential(potential_specs)
    potential.SetParamsState(params_state)

    #read in potential parameter values
    with open('./params_val.json') as data_file:    
        params_val = json.load(data_file)

    #load in xml settings
    xml_extractor = xml_extractor.XMLExtractor()
    xml_extractor.Parse('../settings.xml')

    ###########################################################################################

    optimization_type = xml_extractor.GetText('optimization', 'type').strip().lower()
    if optimization_type == 'relative_entropy':
        from relative_entropy_optimizer import relative_entropy_update
        xmlParser = lambda x: xml_extractor.GetText('optimization', 'relative_entropy', x)

        #perform relative entropy update
        relative_entropy_update = relative_entropy_update.RelativeEntropyUpdate()
        relative_entropy_update.LoadPotential(potential, params_val)
        relative_entropy_update.LoadRadialDistFuncs('./rdf.xvg', '../rdf_target.xvg', spacing=float(xmlParser('dr_integrate')))
        params_val_out, conv_score = relative_entropy_update.CalcUpdate(learning_rate=float(xmlParser('learning_rate')), 
                                                                        conv_crit=xmlParser('conv_crit')
                                                                       )
    else:
        raise Exception('Invalid simulation program provided in settings file.')

    #write out new potential parameter values
    with open('./params_val_out.json', 'w') as data_file:      
        json.dump(params_val_out, data_file, indent=4, sort_keys=True)

    #write out the old one but in sorted order
    with open('./params_val.json', 'w') as data_file:    
        json.dump(params_val, data_file, indent=4, sort_keys=True)
        
    return conv_score

In [8]:
GenerateUpdate()

{'gradient': 48.297530798223058, 'rdf_diff': 263.07838263452385}

In [4]:
import sys
sys.path.insert(0,'../../')

In [6]:
from interface import generate_update

In [7]:
generate_update.GenerateUpdate()

{'gradient': 48.297530798223058, 'rdf_diff': 263.07838263452385}

In [22]:
sys.path

['',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\python27.zip',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\DLLs',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\plat-win',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\lib-tk',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\site-packages',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\site-packages\\Sphinx-1.5.1-py2.7.egg',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\site-packages\\win32',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\rbj428\\AppData\\Local\\Continuum\\Anaconda2_3\\lib\\site-packages\\sputnik-0.9.3-py2.7.egg',

In [1]:
import sys

In [9]:
json?

Object `json` not found.


In [2]:
sys.path.append("C:\\Users\\rbj428\\Documents\\research_projects\\iterative_simulation_framework\\")

In [10]:
"C:\\Users\\rbj428\\Documents\\research_projects\\iterative_simulation_framework\\"

'C:\\Users\\rbj428\\Documents\\research_projects\\iterative_simulation_framework\\'

In [17]:
xml_extractor?

Object `xml_extractor` not found.


In [3]:
bool(False)

False